In [2]:
# NBVAL_SKIP
import os

config = {
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 12,
        "reuse": True,
        }
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    },
    "output_path": "output",

        
}

Convert the Data into Rubix Galaxy HDF5. This will make the call to the IllustrisAPI to download the data, and then convert it into the rubix hdf5 format using the input handler

In [3]:
# NBVAL_SKIP
from rubix.core.data import convert_to_rubix

In [4]:
# NBVAL_SKIP
convert_to_rubix(config)

2024-07-10 15:17:17,726 - rubix - INFO - 
   ___  __  _____  _____  __
  / _ \/ / / / _ )/  _/ |/_/
 / , _/ /_/ / _  |/ /_>  <  
/_/|_|\____/____/___/_/|_|  
                            

2024-07-10 15:17:17,727 - rubix - INFO - Rubix version: 0.0.post65+g022936a.d20240710
2024-07-10 15:17:17,728 - rubix - INFO - Loading data from IllustrisAPI
2024-07-10 15:17:17,728 - rubix - INFO - Reusing existing file galaxy-id-12.hdf5. If you want to download the data again, set reuse=False.
2024-07-10 15:17:17,757 - rubix - INFO - Loading data into input handler
2024-07-10 15:17:17,758 - rubix - DEBUG - Loading data from Illustris file..
2024-07-10 15:17:17,758 - rubix - DEBUG - Checking if the fields are present in the file...
2024-07-10 15:17:17,759 - rubix - DEBUG - Keys in the file: <KeysViewHDF5 ['Header', 'SubhaloData']>
2024-07-10 15:17:17,759 - rubix - DEBUG - Expected fields: ['Header', 'SubhaloData', 'PartType4', 'PartType0']
2024-07-10 15:17:17,759 - rubix - DEBUG - Matching fields: {'

KeyError: "Unable to synchronously open object (object 'Coordinates' doesn't exist)"

In [20]:
# NBVAL_SKIP
from rubix.utils import print_hdf5_file_structure

print(print_hdf5_file_structure("output/rubix_galaxy.h5"))

File: output/rubix_galaxy.h5
Group: galaxy
    Dataset: center (float64) ((3,))
    Dataset: halfmassrad_stars (float64) (())
    Dataset: redshift (float64) (())
Group: meta
    Dataset: BoxSize (float64) (())
    Dataset: CutoutID (int64) (())
    Dataset: CutoutRequest (object) (())
    Dataset: CutoutType (object) (())
    Dataset: Git_commit (|S40) (())
    Dataset: Git_date (|S29) (())
    Dataset: HubbleParam (float64) (())
    Dataset: MassTable (float64) ((6,))
    Dataset: NumFilesPerSnapshot (int64) (())
    Dataset: NumPart_ThisFile (int32) ((6,))
    Dataset: Omega0 (float64) (())
    Dataset: OmegaBaryon (float64) (())
    Dataset: OmegaLambda (float64) (())
    Dataset: Redshift (float64) (())
    Dataset: SimulationName (object) (())
    Dataset: SnapshotNumber (int64) (())
    Dataset: Time (float64) (())
    Dataset: UnitLength_in_cm (float64) (())
    Dataset: UnitMass_in_g (float64) (())
    Dataset: UnitVelocity_in_cm_per_s (float64) (())
Group: particles
    Group

In [21]:
# NBVAL_SKIP
from rubix.utils import load_galaxy_data

load_galaxy_data("output/rubix_galaxy.h5")

({'subhalo_center': array([10410.08408094, 35160.6188816 , 30040.60047238]),
  'subhalo_halfmassrad_stars': 8.711988190754308,
  'redshift': 2.220446049250313e-16,
  'particle_data': {'gas': {'coords': array([[10410.00752676, 35160.60449623, 30040.65473196],
           [10410.03559906, 35160.6237608 , 30040.72992465],
           [10409.99722911, 35160.85307522, 30040.55252916],
           ...,
           [10495.43125783, 35191.46396069, 30133.27693068],
           [10480.20398612, 35238.7471994 , 30165.50465243],
           [10449.15989616, 35239.79823105, 30135.13190636]]),
    'density': array([2.7886430e-02, 4.1865803e-02, 3.8354158e-02, ..., 5.1527832e-06,
           2.3400473e-05, 1.8790468e-05], dtype=float32),
    'electron_abundance': array([1.0563116, 1.0735056, 1.0716248, ..., 1.1896455, 1.106992 ,
           1.1774486], dtype=float32),
    'internal_energy': array([6.8172823e+12, 8.4238611e+12, 8.0589541e+12, ..., 2.3658605e+13,
           4.4322799e+12, 2.2719876e+13], dtyp